### Filtering using scalar subqueries

In [ ]:
SELECT 
-- Select the average of home + away goals, multiplied by 3
	3 * AVG(home_goal + away_goal)
FROM matches_2013_2014;

In [ ]:
SELECT 
	-- Select the date, home goals, and away goals scored
	date,
	home_goal,
	away_goal
FROM matches_2013_2014
-- Filter for matches where total goals exceeds 3x the average
WHERE (home_goal + away_goal) > 
       (SELECT 3 * AVG(home_goal + away_goal)
        FROM matches_2013_2014); 

### Filtering using a subquery with a list

In [ ]:
SELECT 
	-- Select the team long and short names
	team_long_name,
	team_short_name
FROM team
-- Exclude all values from the subquery
WHERE team_api_id NOT IN
     (SELECT DISTINCT hometeam_id FROM match);

Filtering with more complex subquery conditions

In [ ]:
SELECT
	-- Select the team long and short names
	team_long_name,
	team_short_name
FROM team
-- Filter for teams with 8 or more home goals
WHERE team_api_id IN
	  (SELECT hometeam_id 
       FROM match
       WHERE home_goal >= 8);

Subqueries in FROM

In [ ]:
-- This will be the subquery in the main query

SELECT
	t.team_long_name AS team,
	AVG(m.home_goal) AS home_avg
FROM match AS m
LEFT JOIN team AS t
ON m.hometeam_id = t.team_api_id
WHERE season = '2011/2012'
GROUP BY team;

In [ ]:
-- This is the main query with the subquery inside it

SELECT team, home_avg
FROM (SELECT
					t.team_long_name AS team,
					AVG(m.home_goal) AS home_avg
			FROM match AS m
			LEFT JOIN team AS t
			ON m.hometeam_id = t.team_api_id
			WHERE season = '2011/2012'
			GROUP BY team) AS subquery
ORDER BY home_avg DESC
LIMIT 3;

-- Joining subqueries in FROM
-- European Soccer Database

* Construct a subquery that selects only matches with 10 or more total goals.
* Inner join the subquery onto country in the main query.
* Select name from country and count the id column from match.


In [ ]:
SELECT
	-- Select country name and the count match IDs
    c.name AS country_name,
    COUNT(sub.id) AS matches
FROM country AS c
-- Inner join the subquery onto country
-- Select the country id and match id columns
INNER JOIN (SELECT country_id, id 
            FROM match
            -- Filter the subquery by matches with 10+ goals
            WHERE (home_goal + away_goal) >= 10) AS sub
ON c.id = sub.country_id
GROUP BY country_name;

In [ ]:
SELECT
	-- Select country, date, home, and away goals from the subquery
    country,
    date,
    home_goal,
    away_goal
FROM
	-- Select country name, date, home_goal, away_goal, and total goals in the subquery
	(SELECT c.name AS country, 
     	    m.date, 
     		m.home_goal, 
     		m.away_goal,
           (m.home_goal + m.away_goal) AS total_goals
    FROM match AS m
    LEFT JOIN country AS c
    ON m.country_id = c.id) AS subquery
-- Filter by total goals scored in the main query
WHERE total_goals >= 10;